In [1]:
from func.ultis import compressed_pickle
import matplotlib.pyplot as plt

from func.run_pipeline_super_vox import reassign
import numpy as np
from func.plot_utils import colorful_seg, fill_noise

In [9]:
from os import listdir
test_file_names = [f for f in listdir("../data/CellSeg_dataset/HMS_processed/segmentation/")]
test_file_names.remove(".DS_Store")
print(test_file_names)

def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

hand_seg_list = []
raw_img_list = []
boundary_list = []

for i, test_file in enumerate(test_file_names):
    hand_seg = np.load(f"../data/CellSeg_dataset/HMS_processed/segmentation/{test_file}/{test_file}_ins.npy")
    hand_seg_list.append(hand_seg)

    boundary = np.load(f"../data/CellSeg_dataset/HMS_processed/segmentation/{test_file}/{test_file}_boundary_3d_mask.npy")
    boundary_list.append(boundary)

    raw_img = np.load(f"../data/CellSeg_dataset/HMS_processed/raw/{test_file}.npy")
    raw_img_list.append(normalize_data(raw_img))

['135', '95', '50', '150', '195', '105', '60', '160', '220', '210', '205', '180', '145', '80', '120', '175', '75', '110', '165', '65', '100', '190', '155', '55', '130', '90', '215', '225', '200', '115', '70', '170', '85', '125', '140', '185']


# Calculate SSIM

In [17]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm.notebook import tqdm

ssim_list = []
psnr_list = []

for i in tqdm(range(len(raw_img_list))):
    ssim_list.append(ssim(raw_img_list[i], boundary_list[i], channel_axis=None))
    psnr_list.append(psnr(raw_img_list[i], boundary_list[i], data_range=1))

  0%|          | 0/36 [00:00<?, ?it/s]

In [18]:
# save it
compressed_pickle("ssim_list_HMS", ssim_list)
compressed_pickle("psnr_list_HMS", psnr_list)

# Calculate the number of cell instances and their size

In [15]:
value_count_list = []
for i in tqdm(range(len(hand_seg_list))):
    value_count_list.append(np.unique(hand_seg_list[i], return_counts=True))

  0%|          | 0/36 [00:00<?, ?it/s]

In [16]:
# save it
compressed_pickle("value_count_list_HMS", value_count_list)

In [22]:
# background percentage:
background_percentages_list = []

for hand_seg in hand_seg_list:
    background_size = np.count_nonzero(hand_seg==0.)
    total_size = hand_seg.size
    background_percentages_list.append(background_size/total_size)